In [2]:
# This makes sure you can run this notebook multiple times without errors
import os
try:
    os.remove('airline2.db')
except OSError:
    pass

In [3]:
import sqlite3
conn = sqlite3.connect('airline2.db')

In [4]:
import pandas as pd

airports = pd.read_csv("../dataverse_files/airports.csv")
carriers = pd.read_csv("../dataverse_files/carriers.csv")
planes = pd.read_csv("../dataverse_files/plane-data.csv")

In [ ]:
df_list = []
for i in range(2000,2006):
    f_path = "../dataverse_files/ontime/{}.csv".format(str(i))
    print(f_path)
    temp_df = pd.read_csv(f_path, engine='python',encoding="ISO-8859-1")
    df_list.append(temp_df)
ontime = pd.concat(df_list)

../dataverse_files/ontime/2000.csv


In [ ]:
# index = False to ensure the DataFrame row index is not written into the SQL tables
airports.to_sql('airports', con = conn, index = False)
carriers.to_sql('carriers', con = conn, index = False)
planes.to_sql('planes', con = conn, index = False)
ontime.to_sql('ontime', con = conn, index = False)

In [ ]:
# practice
# Q1: Which of the following companies has the highest number of cancelled flights, relative to their number of total flights?
targets = ['United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.']
sql_str = """
 SELECT
  `Description`,
  `n.total`,
  `n.cancelled`,
  ((`n.cancelled` * 100.0) / `n.total`) AS `percent_c`
FROM (
  SELECT `carriers`.*, `n.total`, `n.cancelled`
  FROM `carriers`
  INNER JOIN (
    SELECT
      `LHS`.`UniqueCarrier` AS `UniqueCarrier`,
      `LHS`.`n` AS `n.total`,
      `RHS`.`n` AS `n.cancelled`
    FROM (
      SELECT `UniqueCarrier`, COUNT(*) AS `n`
      FROM `ontime`
      GROUP BY `UniqueCarrier`
    ) AS `LHS`
    INNER JOIN (
      SELECT `UniqueCarrier`, COUNT(*) AS `n`
      FROM `ontime`
      WHERE (`Cancelled` = 1.0)
      GROUP BY `UniqueCarrier`
    ) AS `RHS`
      ON (`LHS`.`UniqueCarrier` = `RHS`.`UniqueCarrier`)
  ) AS `RHS`
    ON (`carriers`.`Code` = `RHS`.`UniqueCarrier`)
)
WHERE (`Description` IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.'))
ORDER BY `percent_c` DESC
LIMIT 1
"""

c = conn.cursor()
import pandas as pd
q = c.execute(sql_str).fetchall()
df = pd.DataFrame(q)
df.to_csv("q1.csv")

In [ ]:
# practice
# Q2:Which of the following cities has the highest number of inbound flights (excluding cancelled flights)?
sql_str = """
SELECT `city`, SUM(`n_total`) AS `n`
FROM (
  SELECT `city`, `n_total`
  FROM `airports`
  INNER JOIN (
    SELECT `Dest`, COUNT(*) AS `n_total`
    FROM `ontime`
    WHERE (`Cancelled` = 0.0)
    GROUP BY `Dest`
  ) AS `RHS`
    ON (`airports`.`iata` = `RHS`.`Dest`)
)
GROUP BY `city`
ORDER BY `n` DESC
LIMIT 1
"""
c = conn.cursor()
import pandas as pd
q = c.execute(sql_str).fetchall()
df = pd.DataFrame(q)
df.to_csv("q2.csv")

In [ ]:
# Q4:Which of the following airplanes has the lowest associated average departure delay (excluding cancelled and diverted flights)?
targets = ['737-230', 'ERJ 190-100 IGW', 'A330-223', '737-282']
sql_str = """
SELECT `model`, `n_avg`
FROM (
  SELECT `planes`.*, `n_avg`
  FROM `planes`
  INNER JOIN (
    SELECT `TailNum`, AVG(`DepDelay`) AS `n_avg`
    FROM `ontime`
    WHERE (`Cancelled` = 0.0 OR `Diverted` = 0.0)
    GROUP BY `TailNum`
  ) AS `RHS`
    ON (`planes`.`tailnum` = `RHS`.`TailNum`)
)
WHERE (`model` IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.'))
ORDER BY `n_avg`
LIMIT 1
"""
c = conn.cursor()
import pandas as pd
q = c.execute(sql_str).fetchall()
df = pd.DataFrame(q)
df.to_csv("q4.csv")

In [ ]:
# Q5:Which of the following companies has the highest number of cancelled flights?
sql_str = """
SELECT `Description`, `n.cancelled`
FROM (
  SELECT `carriers`.*, `n.cancelled`
  FROM `carriers`
  INNER JOIN (
    SELECT `UniqueCarrier`, COUNT(*) AS `n.cancelled`
    FROM `ontime`
    WHERE (`Cancelled` = 1.0)
    GROUP BY `UniqueCarrier`
  ) AS `RHS`
    ON (`carriers`.`Code` = `RHS`.`UniqueCarrier`)
)
WHERE (`Description` IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.'))
ORDER BY `n.cancelled` DESC
LIMIT 1
"""
c = conn.cursor()
import pandas as pd
q = c.execute(sql_str).fetchall()
df = pd.DataFrame(q)
df.to_csv("q5.csv")

# Manipulate Databases Using Python

In [ ]:
c = conn.cursor()
c.execute('''
SELECT name
FROM sqlite_master
WHERE type='table'
''')
c.fetchall()
import pandas as pd
q = c.execute("SELECT * FROM Student").fetchall()
pd.DataFrame(q)

# Add a New Table

In [ ]:
c.execute('''
CREATE TABLE Teacher (staff_id TEXT PRIMARY KEY,
name TEXT)
''')

In [ ]:
conn.commit() # save (commit) the changes

In [ ]:
c.execute('''
SELECT name
FROM sqlite_master
WHERE type='table'
''').fetchall()

# Delete a Table

In [ ]:
c.execute("DROP TABLE Teacher")

In [ ]:
conn.commit()

In [ ]:
c.execute('''
SELECT name
FROM sqlite_master
WHERE type='table'
''').fetchall()

In [ ]:
c.execute('''
SELECT name
FROM sqlite_master
WHERE type='table'
''').fetchall()

# Insert Tuples/Rows

In [ ]:
c.execute("INSERT INTO Student VALUES(202029744, 'Harper Taylor', 1)")

In [ ]:
conn.commit()

In [ ]:
q = c.execute("SELECT * FROM Student").fetchall()
pd.DataFrame(q)

# Update Tuples/Rows

In [ ]:
c.execute('''
UPDATE Student
SET student_id = "201929744"
WHERE name = "Harper Taylor"
''')

In [ ]:
conn.commit()

In [ ]:
q = c.execute("SELECT * FROM Student").fetchall()
pd.DataFrame(q)

# Delete Tuples/Rows

In [ ]:
c.execute('''
DELETE FROM Student
where name = "Harper Taylor"
''')

In [ ]:
conn.commit()

In [ ]:
q = c.execute("SELECT * FROM Student").fetchall()
pd.DataFrame(q)

# Disconnecting From the Database

In [ ]:
conn.close()